In [2]:
from sklearn.inspection import permutation_importance
import time 

from Script import *

df_train = pd.read_csv("Data/train.csv")
df_test  = pd.read_csv("Data/test.csv")

df_train = df_train.drop("i", axis=1)
y = df_train.pop("y")
df_train.insert(54,"y", y)
df_test = df_test.drop(["y", "i"], axis = 1)


df_train, df_test = preprocess_data(df_train, df_test, verbose=False)


X = df_train.iloc[:, 0:-1]
y = df_train.iloc[:, -1]


In [11]:

import joblib
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score


rf_grid_search = joblib.load("Temp/rf_grid_search.joblib")

clf1 = rf_grid_search.best_estimator_



clf2 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0)

print("Fitting Classifiers")
eclf1 = VotingClassifier(estimators=[
        ('rf', clf1), ('xgb', clf2)], voting='hard')


clf = BaggingClassifier(base_estimator=eclf1,
                        n_estimators=10, random_state=0)

Fitting Classifiers


In [12]:
clf

BaggingClassifier(base_estimator=VotingClassifier(estimators=[('rf',
                                                               RandomForestClassifier(max_depth=60,
                                                                                      max_features='sqrt',
                                                                                      min_samples_leaf=5,
                                                                                      min_samples_split=6,
                                                                                      n_estimators=1900)),
                                                              ('xgb',
                                                               GradientBoostingClassifier(learning_rate=1.0,
                                                                                          max_depth=1,
                                                                                          random_state=0))]),
            

In [20]:







print("Hard Voting ")
hard_scores = cross_val_score(clf, X, y, cv=3) #try 3 better
clf.fit(X,y)

print("aux stop")



# name_of_model = "bagging_rf_best+xgboost_correct_types"

# joblib.dump(hard_scores, "Temp/"+name_of_model+"_hard_scores"+".joblib")


print("hard voting", hard_scores.mean())


# joblib.dump(eclf1, "Temp/"+name_of_model+".joblib")




Hard Voting 
aux stop
hard voting 0.6877572016460904


In [21]:
score_training = clf.score(X,y)
print(score_training)
name_of_model = "bagging_rf_best+xgboost_correct_types"

0.7155349794238683


In [22]:


preds_hard = clf.predict(df_test)
save_predictions(preds_hard, "Predictions/"+name_of_model+"_hard")

In [ ]:
name_of_model = "bagging_rf_best+xgboost_correct_types"
joblib.dump(hard_scores, "Temp/"+name_of_model+"_hard_scores"+".joblib")
joblib.dump(eclf1, "Temp/"+name_of_model+".joblib")